
### <font color="Red" size="6"> Juan Esteban Moreno Agudelo</font>
#***Solución del problema 3 de la tarea***

### <font color="blue" size="7"> 
                  1.Verbalización 
</font>

### Es sobre una empresa que produce alimento para animales de campo, tiene 3 plantas de producción, tiene un conjunto de clientes y además los 5 productos que produce estan consitituidos por la disponibilidad de otras materias primas.

## <font color="blue"> 1.1.Objetivo del problema </font>
### Minimizar los costos de transporte de los productos desde cada planta hacia cada cliente cumpliendo con la demanda requerida.

## <font color="blue"> 1.2. Decisiones del problema </font>
* ### Qué productos y en qué cantidad se debe producir en cada una de las plantas ?, Qué productos y en qué cantidad deben ser enviados a cada cliente desde cada una de las plantas? 

## <font color="blue"> 1.3. Restricciones del problema </font>
* ### La producción de productos de una planta no puede superar su capacidad máxima
* ### Se debe garantizar que se cumpla con la composición de materias primas de dentro de cada planta, que no sobrepase de la cantidad que dispone.
* ### Garantizar que se cumpla con la demanda solicitada por cada cliente.


### <font color="blue" size="7"> 
                2.Modelación matemática  
</font>

### Se escribirá y detallará todo lo requerido para expresar matematicamente el problema.

### <font color="blue"> 2.1. Conjuntos </font>
> ### $F_i:$ *Conjunto de fábricas*
> ### $C_j:$ *Conjunto de clientes*
> ### $M_k:$ *Conjunto de materias primas*
> ### $P_n:$ *Conjunto de productos*


### <font color="blue"> 2.2. Parámetros </font>
> ### $cm_i = \,$ *Capacidad máxima de la fábrica $i$.*

> ### $dem_{jn} = \,$ *Demanda requerida del cliente $j$ del producto $n$* 

> ### $mp_{kn} = \,$ *Contenido de la materia prima $k$ en el producto $n$*

> ### $dmp_{ik} = \,$ *Disponibilidad en la fábrica $i$ de la materia prima $k$*

> ### $c_{ij} = \,$ *Costo de transporte desde la fábrica $i$ hacia el cliente $j$*

### <font color="blue"> 2.3. Variables de decisión </font>

> ### $x_{ijn} = \,$ *Costo de transporte desde la fábrica $i$ hacia el cliente $j$ del producto $n$.*

### <font color="blue"> 2.4. Función objetivo </font>
### Minimizar el costo de transporte de lo productos hacia los clientes.

### $$argmin\sum_{\ n \in P} \sum_{\ i \in F} \sum_{\ j \in C} x_{ijn}c_{ij} 
$$


### <font color="blue"> 2.5. Restricciones </font>
* ### 2.5.1. La producción de una fábrica no puede superar su capacidad. 

### $$ \sum_{\ j \in C} \sum_{\ n \in P} x_{ijn} \leq cm_i\,\,\,\,\ \forall i \in F$$   

* ### 2.5.2. Restricción de disponibilidad de materia prima en las fábricas.
 
### $$ \sum_{\ j \in C} \sum_{\ n \in P} (x_{ijn})mp_{kn} \leq dmp_{ik}\,\,\,\,\ \forall i \in F, \,\,\, \forall k \in M$$   

* ### 2.5.3. Restricción de satisfacción de la demanda de cada cliente.

### $$ \sum_{\ i \in F} x_{ijn} == dem_{jn} \,\,\, \forall j \in C, \,\,\, \forall n \in P$$

* ### 2.5.4. Restricción de dominio de las variables.
### $$ x_{ijn} \in  R  \,\,\,\,\,\, \forall i \in F,\,\, \forall j \in C,\,\,\forall n \in P$$
 

### <font color="blue" size="7">  
               3.Alistando instancias necesarias  
</font>


## <font color="blue"> 3.1. Importación de paquetes </font>




In [ ]:
!pip install pyomo 
!pip install matplotlib
!pip install seaborn
import pyomo.environ as pyo
from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition
import pandas as pd
import numpy as np
import copy

     |████████████████████████████████| 9.4MB 3.2MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 256kB 43.9MB/s 
     |████████████████████████████████| 163kB 45.0MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/My Drive/Colab Notebooks/"

# Traigo los datos
demanda_cliente = pd.read_csv(path + "demanda_cliente.csv", index_col=0)
disponibilidad_mp = pd.read_csv(path + "disponibilidad_mp.csv", index_col=0)
costo_transp = pd.read_csv(path + "costo_transporte.csv", index_col=0)
composi_prod = pd.read_csv(path + "composicion_producto.csv", index_col=0)

In [ ]:
costo_transp

,P1,P2,P3
cliente,,,
c1,2,20,6
c2,16,38,19
c3,44,40,20
c4,98,27,73
c5,50,97,58
c6,57,8,15
c7,91,84,60
c8,94,50,27
c9,65,21,93


In [ ]:
model = pyo.ConcreteModel()
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT_EXPORT)
model.rc = pyo.Suffix(direction=pyo.Suffix.IMPORT_EXPORT)

# conjuntos
clientes = list(demanda_cliente.index)
productos = demanda_cliente.columns.values.tolist()
plantas = costo_transp.columns.values.tolist()
mat_prima = disponibilidad_mp.columns.values.tolist()

In [ ]:
# Pyomo sets
model.clientes = pyo.Set(initialize=clientes, ordered=False, doc="Conjunto de clientes")
model.productos = pyo.Set(initialize=productos, ordered=False, doc="Conjunto de productos")
model.fabricas = pyo.Set(initialize=plantas, ordered=False, doc="Conjunto de fábricas")
model.mat_prima = pyo.Set(initialize=mat_prima, ordered=False, doc="Conjunto de materias primas")

In [ ]:
##### Parámetros #####

# 1.Capacidades máximas
capacidad_max = {"P1": 52, "P2": 50, "P3": 60}
model.cm = pyo.Param(model.fabricas, initialize=capacidad_max, 
                     domain=pyo.NonNegativeIntegers, doc="capacidades máximas")

# 2. Demanda por cliente
dem_cli = {(i, j): demanda_cliente.loc[i, j] for i in clientes for j in productos}
model.dem = pyo.Param(model.clientes, model.productos, 
                      initialize=dem_cli, domain=pyo.NonNegativeReals, doc="Demanda de cada cliente (toneladas)")

# 3. Contenido de materia prima
cont_mp = {(i, j): composi_prod.loc[i, j] for i in model.productos for j in model.mat_prima}
model.mp = pyo.Param(model.productos, model.mat_prima, initialize=cont_mp,
                     domain=pyo.NonNegativeReals, doc="Composición de cada producto")

# 4. Disponibilidad de materias primas en la fábrica
disp_mp = {(i, j): disponibilidad_mp.loc[i, j] for i in model.fabricas for j in model.mat_prima}
model.dmp = pyo.Param(model.fabricas, model.mat_prima, 
                        initialize=disp_mp, doc="Disponibilidad de materias primas en las plantas")
# 5. Costo de transporte
costo = {(i, j): costo_transp.loc[i, j] for i in model.clientes for j in model.fabricas}
model.c = pyo.Param(model.clientes, model.fabricas, domain=pyo.NonNegativeIntegers,
                    initialize=costo, doc="Costo de transporte entre clientes y plantas")

In [ ]:
##### Variables de decisión #####
model.x = pyo.Var(model.fabricas, model.clientes, model.productos, 
                  within=pyo.NonNegativeReals, 
                  doc="Cantidad de producto que es enviado desde determinada fábrica hacia algun cliente")

In [ ]:
# Funcion objetivo
def obj_function(model):
    return sum(sum(sum(model.x[i,j,n]*model.c[j, i] for i in model.fabricas) for j in model.clientes) for n in model.productos)

model.objective = pyo.Objective(rule=obj_function, sense=pyo.minimize,
                                doc="cost function")

In [ ]:
# Constraints
# 1. La producción de una fábrica no puede superar su capacidad.

def constraint_1(model, i):
    return sum(sum(model.x[i,j,n] for n in model.productos) for j in model.clientes) <= model.cm[i]
model.r1 = pyo.Constraint(model.fabricas, rule=constraint_1)

# 2. Restricción de disponibilidad de materia prima en las fábricas
def constraint_2(model, i, k):
    return sum(sum(model.mp[n, k]*model.x[i, j, n] for n in model.productos) for j in model.clientes) <= model.dmp[i, k]
model.r2 = pyo.Constraint(model.fabricas, model.mat_prima, rule=constraint_2)

# 3. Restricción de la satisfacción de demanda.
def constraint_3(model, j, n):
    return sum(model.x[i,j,n] for i in model.fabricas) == model.dem[j, n]
model.r3 = pyo.Constraint(model.clientes, model.productos, rule=constraint_3)

In [ ]:
# instal cbc
!apt-get install -y -qq coinor-cbc

In [ ]:
# Nos conectamos al servidor NEOS

# 1. Método 1: Instalando solver instalado de forma local
opt = pyo.SolverManagerFactory('neos')
results = opt.solve(model, opt='cplex', tee=True)

# 2 Método 2: Ejecutando un solver disponible
opt = pyo.SolverFactory("cbc", executable='/usr/bin/cbc')
results = opt.solve(model)  

In [ ]:
# Valor de la variable solución.
model.x.display()

x : Cantidad de producto que es enviado desde determinada fábrica hacia algun cliente
    Size=150, Index=x_index
    Key                : Lower : Value       : Upper : Fixed : Stale : Domain
     ('P1', 'c1', 'A') :     0 :         0.3 :  None : False : False : NonNegativeReals
     ('P1', 'c1', 'B') :     0 :         2.0 :  None : False : False : NonNegativeReals
     ('P1', 'c1', 'C') :     0 :         1.9 :  None : False : False : NonNegativeReals
     ('P1', 'c1', 'D') :     0 :         2.7 :  None : False : False : NonNegativeReals
     ('P1', 'c1', 'E') :     0 :         2.2 :  None : False : False : NonNegativeReals
    ('P1', 'c10', 'A') :     0 :         0.0 :  None : False : False : NonNegativeReals
    ('P1', 'c10', 'B') :     0 :         0.0 :  None : False : False : NonNegativeReals
    ('P1', 'c10', 'C') :     0 :         0.0 :  None : False : False : NonNegativeReals
    ('P1', 'c10', 'D') :     0 :         0.0 :  None : False : False : NonNegativeReals
    ('P1', 'c10'

In [ ]:
for i in model.fabricas:
    print(f"Holguras de plantas (restricción 1) {i}: {model.r1[i].uslack()}")

Holguras de plantas (restricción 1) P2: 7.105427357601002e-15
Holguras de plantas (restricción 1) P1: 16.335714281
Holguras de plantas (restricción 1) P3: 25.664285686000007


In [ ]:
for i in model.fabricas:
    for j in model.mat_prima:
        print(f"Holguras de materia prima en plantas (restricción 2) {i, j}: {model.r2[i, j].uslack()}")

Holguras de materia prima en plantas (restricción 2) ('P2', 'MP3'): 4.360000000000001
Holguras de materia prima en plantas (restricción 2) ('P2', 'MP2'): 1.7763568394002505e-15
Holguras de materia prima en plantas (restricción 2) ('P2', 'MP1'): 0.6400000000000041
Holguras de materia prima en plantas (restricción 2) ('P1', 'MP3'): 10.395714284
Holguras de materia prima en plantas (restricción 2) ('P1', 'MP2'): 3.9399999981999994
Holguras de materia prima en plantas (restricción 2) ('P1', 'MP1'): -1.2000000992884452e-09
Holguras de materia prima en plantas (restricción 2) ('P3', 'MP3'): 1.5142857144000015
Holguras de materia prima en plantas (restricción 2) ('P3', 'MP2'): -5.599998686989238e-09
Holguras de materia prima en plantas (restricción 2) ('P3', 'MP1'): 9.149999977200002


In [ ]:
# Variables duales
print("Duals")
for c in model.component_objects(pyo.Constraint, active=True):
    print("Constraint:", c)
    for index in c:
        print(f"{index}: {model.dual[c[index]]}")

Duals
Constraint: r1
P2: -7.0
P1: 0.0
P3: 0.0
Constraint: r2
('P2', 'MP3'): 0.0
('P2', 'MP2'): -70.714286
('P2', 'MP1'): 0.0
('P1', 'MP3'): 0.0
('P1', 'MP2'): 0.0
('P1', 'MP1'): -21.428571
('P3', 'MP3'): 0.0
('P3', 'MP2'): -70.714286
('P3', 'MP1'): 0.0
Constraint: r3
('c2', 'C'): 33.142857
('c2', 'E'): 20.285714
('c2', 'B'): 28.857143
('c2', 'A'): 26.714286
('c2', 'D'): 22.428571
('c4', 'C'): 48.142857
('c4', 'E'): 62.285714
('c4', 'B'): 55.214286
('c4', 'A'): 51.678571
('c4', 'D'): 55.214286
('c3', 'C'): 34.142857
('c3', 'E'): 48.285714
('c3', 'B'): 0.0
('c3', 'A'): 37.678571
('c3', 'D'): 41.214286
('c5', 'C'): 67.142857
('c5', 'E'): 54.285714
('c5', 'B'): 62.857143
('c5', 'A'): 60.714286
('c5', 'D'): 0.0
('c7', 'C'): 0.0
('c7', 'E'): 88.285714
('c7', 'B'): 81.214286
('c7', 'A'): 77.678571
('c7', 'D'): 0.0
('c1', 'C'): 19.142857
('c1', 'E'): 6.2857143
('c1', 'B'): 14.857143
('c1', 'A'): 12.714286
('c1', 'D'): 8.4285714
('c8', 'C'): 41.142857
('c8', 'E'): 55.285714
('c8', 'B'): 48.2142

In [ ]:
# Costos reducidos de las variables.
print("Variables")
for c in model.component_objects(pyo.Var, active=True):
    print("Variables:", c)
    for index in c:
        print(f"{index}: {model.rc[c[index]]}")

Variables
Variables: x
('P2', 'c2', 'C'): 26.0
('P2', 'c2', 'E'): 53.0
('P2', 'c2', 'B'): 37.357143
('P2', 'c2', 'A'): 35.964286
('P2', 'c2', 'D'): 43.785714
('P2', 'c4', 'C'): 0.0
('P2', 'c4', 'E'): 0.0
('P2', 'c4', 'B'): 0.0
('P2', 'c4', 'A'): 0.0
('P2', 'c4', 'D'): 0.0
('P2', 'c3', 'C'): 27.0
('P2', 'c3', 'E'): 27.0
('P2', 'c3', 'B'): 68.214286
('P2', 'c3', 'A'): 27.0
('P2', 'c3', 'D'): 27.0
('P2', 'c5', 'C'): 51.0
('P2', 'c5', 'E'): 78.0
('P2', 'c5', 'B'): 62.357143
('P2', 'c5', 'A'): 60.964286
('P2', 'c5', 'D'): 125.21429
('P2', 'c7', 'C'): 105.14286
('P2', 'c7', 'E'): 31.0
('P2', 'c7', 'B'): 31.0
('P2', 'c7', 'A'): 31.0
('P2', 'c7', 'D'): 112.21429
('P2', 'c1', 'C'): 22.0
('P2', 'c1', 'E'): 49.0
('P2', 'c1', 'B'): 33.357143
('P2', 'c1', 'A'): 31.964286
('P2', 'c1', 'D'): 39.785714
('P2', 'c8', 'C'): 30.0
('P2', 'c8', 'E'): 30.0
('P2', 'c8', 'B'): 30.0
('P2', 'c8', 'A'): 30.0
('P2', 'c8', 'D'): 30.0
('P2', 'c9', 'C'): 0.0
('P2', 'c9', 'E'): 0.0
('P2', 'c9', 'B'): 0.0
('P2', 'c9', 

> ### <font color="red"> Cambio en la función objetivo 

### 1. ¿Cuál sería el impacto sobre la función objetivo de atender la totalidad de la demanda del cliente 10 (Todos  los productos) desde la planta número 2? 

### <font color="gray"> *R/ Es como imponerle una nueva restricción al modelo, si la cumple seguimos en el óptimo y el valor de la función objetivo no ha cambiado. Podría pasar que esta nueva restricción me cambie el punto óptimo y deba encontrar un nuevo espacio de solución* </font>

> ### <font color="red"> Cambio en la función objetivo 

### 2. Teniendo en cuenta que la planta 2 ya agotó toda su capacidad ¿desde qué planta enviaría una tonelada  adicional de producto E al cliente 4? justifique su respuesta.

### <font color="gray"> *R/ Buscaría la planta que tenga mayor holgura ya que tengo certeza de que puedo aumentar la producción del producto requerido y además elegiría entre ellas la más barata.* </font>
* ### Holguras P1: 14.335714281000001  
* ### Holguras P3: 25.664285686
* ### Holguras P2: 0.0

### <font color="gray"> *En este caso las plantas 1 y 3 tienen holgura, ya luego miro cual es más barata. Los parámetros me dicen que la planta # 3 es la más barata, entonces decidiría enviar la tonelada extra desde allí.*</font>
* ### ('c4', 'P1') :    98
* ### ('c4', 'P2') :    27
* ### ('c4', 'P3') :    73

> ### <font color="red"> Cambio en la función objetivo 

### 3. ¿Cuál sería el precio máximo que usted estaría dispuesto a pagar por enviar producto C de la planta 1 al cliente  1?  

### <font color="gray"> *Estoy en un problema de minimización, así que si comienzo a incrementar ese costo de transporte, empeoraré la F.O ya que esa variable es básica. Asi que no estoy dispuesto a pagar un peso extra de transporte para mover esa carga, el precio máximo son 2 dólares.* </font>

> ###  <font color="red"> Cambio en el coeficiente de una variable no-básica en la función objetivo. 

### 4. Un nuevo proveedor de transporte le ofrece llevar producto E, desde la    planta 1 hasta el cliente 9 por $45 la  tonelada, ¿es atractiva esta oferta para usted? 

### <font color="gray"> *R/ Si, porque al evaluar los costos reducidos de esa variable su valor inicial es de 13, y el coeficiente inicial de la variable no básica es 65 que corresponde al costo de transporte, pero me dicen que alguien ofrece bajarlo a 45, es decir hay un cambio de 20, como el cambio es mayor que el costo reducido, la haría atractiva. Esta variable entrará a la base y su costo reducido será 0, estaríamos parados en una solución óptima alterna.* </font>

> ###  <font color="red"> Cambio en el coeficiente de una variable básica en la función objetivo. 

### 5. El proveedor encargado de transportar los productos E, desde la planta 3 hasta el cliente 6 ha decidido aumentar el precio de su flete de 15dolares a 20dolares, ¿Afecta esto la optimalidad de la solución actual?

### <font color="gray"> *El costo reducido no se ve afectado por ese cambio en el coeficiente, sigue siendo 0. La optimalidad del problema no se ve afectada, al incrementar ese costo lo que ocurre es que ya el modelo buscará suplir esa demanda desde una planta en la cual el transporte sea más barato alterando las variables no-básicas. El modelo cuando se soluciona señala lo siguiente:* </font>
* ###  *(P3, c6, E)=0*
* ### *costo reducido de esa variable es: (P3, c6, E)= 2.1316282e-14*
* ### *Con 15dol se tiene F.0=2973.5714 y con 20dol se tiene F.0=2987.3412*

### <font color="gray"> *El cambio en la función objetivo se debió muy seguramente a cambios en alguna variable no-básica.*</font>


> ###  <font color="red"> Cambio de parámetros, incremento en la derecha de una restricción. 

### 6. Su proveedor ha decidido ofrecerle una tonelada más de MP1 por $15, ¿en cuál de las plantas de producción sería atractivo comprar esta tonelada de materia prima?

### <font color="gray"> *Si requiero esa tonelada de MP1, lo que evaluaría es la holgura en las plantas para poder saber cual de ellas esta en condiciones de producir esa tonelada extra y mirando las variables del problema dual. Se tiene lo siguiente:* </font>
* ### **Holgura (P3, MP1)=9.149999977199997**
* ### **Valor de la variable dual (P3, MP1)=0.0**

### <font color="gray"> *La variable dual me dice que no incurriré en costos sobre la F.O si incremento el lado derecho de esa restricción en el primal hasta 9.1499 en la planta 3, así que la respuesta es que solicitaría esa tonelada a dicha planta.*</font>

> ###  <font color="red"> Cambio de parámetros, incremento en la derecha de una restricción. 

### 7. Si pudiera comprar una tonelada más de cualquier materia prima, ¿Cuál compraría y que planta la utilizaría?

### <font color="gray"> *Tendría que evaluar las holguras de la restricciones de materia prima en las plantas como se muestran acontinuación.* </font>

* ### Holguras (P3, MP3): 1.5142857143999997
* ### Holguras (P3, MP1): 9.1499999772
* ### Holguras (P3, MP2): -5.600000463346078e-09
* ### Holguras (P2, MP3): 4.359999999999999
* ### Holguras (P2, MP1): 0.639999999999997
* ### Holguras (P2, MP2): 1.7763568394002505e-15
* ### Holguras (P1, MP3): 10.395714283999999
* ### Holguras (P1, MP1): -1.1999983229316058e-09
* ### Holguras (P1, MP2): 3.939999998199999

### <font color="gray"> *Si vemos por ejemplo la planta 3 tiene holgura para la materia prima 1 y 3 pero no para la 2, así que no podría pedirle "cualquiera", y así ocurre con el resto de plantas, es decir, no hay forma de que de una misma planta pueda pedir cualquiera de las materias primas. En este caso compraría materia prima 3 de la planta 1, porque es la holgura mayor y el costo por producirla no afectará la F.O., esta información me la entrega la variable dual de esa restricción, que en este caso es 0.*</font>



> ###  <font color="red"> Cambios en la parte derecha de una restrcción.

### 8. ¿Cuánto estaría dispuesto a pagar por aumentar la capacidad de producción de cada una de las plantas?

### <font color="gray"> *Si miramos las variables duales asociadas a la capacidad de producción de las plantas, que son las siguientes:* </font>
* ### **P3 = 0.0**
* ### **P2 = 7.0**
* ### **P1 = 0.0**

### <font color="gray"> *El precio que estaría dispuesto a pagar por aumentar la producción en cada planta sería el asociado a la planta 2 porque en las otras cuento con holgura. El precio a pagar será (7 por cada unidad extra en esa planta)*</font>

> ###  <font color="red"> Cambios en el lado derecho de las restricciones.

### 9. ¿Hasta qué punto es posible aumentar la capacidad de producción de las plantas sin que se afecte la optimalidad de la solución actual?

### <font color="gray"> *Es posible aumentar la capacidad de producción de las planta hasta que me gaste la holgura.* </font>
* ### **Holguras P3 = 25.664285686**
* ### **Holguras P2 = 7.105427357601002e-15**
* ### **Holguras P1 = 16.335714281**

### <font color="gray"> *La planta 2 esta practicamente a tope, así que de la única forma que no afectaría la optimalidad sería aumentando la planta 3 y la planta 1, podemos ver que en el dual en la planta 2, aumentar una unida extra aumentaría el costo de la F.O.*</font>
* ### **P3 = 0.0**
* ### **P2 = -7.0**
* ### **P1 = 0.0**


> ###  <font color="red"> Cambio en el lado derecho de una restricción.

### 10. El cliente 3 desea saber ¿cuánto debería pagar por una tonelada adicional de producto D?

### <font color="gray"> *Para esto, debemos mirar la variable dual asociada a esa restricción, el valor es el siguiente:* </font>
* ### **(c3, D) = 41.214286**

### <font color="gray"> *El precio que estaría dispuesto a pagar por una unidad extra sería 41.214286 dolares.*</font>


In [ ]:
print(f"Valor de la función objetivo {model.objective.expr()}")

Valor de la función objetivo 2973.571429304001
